In [9]:
import sys
import json
import pandas as pd
from pandas.io.json import json_normalize
sys.path.append("../") # Enables importing from parent directory
import fpl_draft_league as fpl

In [6]:
# Pull the transactions API data into json
fpl.get_json('transactions.json',
         'https://draft.premierleague.com/api/draft/league/38996/transactions',
        'lee.gower17@gmail.com') 

Enter Password:  ········


In [13]:
fpl.get_json('elements.json',
            'https://draft.premierleague.com/api/bootstrap-static',
            'lee.gower17@gmail.com')

Enter Password:  ········


In [23]:
fpl.get_json('details.json',
            'https://draft.premierleague.com/api/league/38996/details',
            'lee.gower17@gmail.com')

Enter Password:  ········


In [22]:
def json_to_df(json_file, tables):
    
    with open(json_file) as json_data:
        d = json.load(json_data)
        transactions_df = json_normalize(d['transactions'])
    
    return transactions_df

SyntaxError: invalid syntax (<ipython-input-22-132e5adea798>, line 9)

In [28]:
transactions_df = json_to_df('transactions.json')

In [19]:
with open('elements.json') as json_data:
        d = json.load(json_data)
        elements_df = json_normalize(d['elements'])
        
elements_df[elements_df['id'] == 147]

,added,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,creativity,draft_rank,...,red_cards,saves,second_name,squad_number,status,team,threat,total_points,web_name,yellow_cards
146,2019-06-20T16:48:00.005055Z,0,0,42,100.0,100.0,1,66838,49.1,118,...,0,0,Tosun,None,a,7,167.0,13,Tosun,0


In [20]:
elements_df[elements_df['id'] == 26]

,added,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,creativity,draft_rank,...,red_cards,saves,second_name,squad_number,status,team,threat,total_points,web_name,yellow_cards
25,2019-06-20T16:47:59.869843Z,0,0,14,50.0,50.0,0,54764,12.2,214,...,0,0,Kodjia,None,d,2,24.0,5,Kodjia,1


## Total transactions by player
I've got the transactions_df, and can get the league_entries_df, so just need to join these to and then do some aggregation.

In [24]:
with open('details.json') as json_data:
        d = json.load(json_data)
        entries_df = json_normalize(d['league_entries'])

In [31]:
entries_df = entries_df[['entry_id', 'player_first_name']]

In [29]:
transactions_df.head()

,added,element_in,element_out,entry,event,id,index,kind,priority,result
0,2019-08-11T19:35:07.445403Z,487,115,176805,2,1204474,2.0,w,1.0,di
1,2019-08-11T19:35:07.445504Z,495,387,176805,2,1204475,3.0,w,2.0,a
2,2019-08-11T19:35:07.445575Z,42,186,176805,2,1204476,21.0,w,3.0,a
3,2019-08-11T19:35:07.445644Z,4,186,176805,2,1204477,37.0,w,4.0,di
4,2019-08-11T19:35:07.445712Z,429,387,176805,2,1204478,38.0,w,5.0,do


In [32]:
df = pd.merge(transactions_df, entries_df, how='left', left_on='entry', right_on='entry_id')

In [36]:
df = df[df['result'] == 'a']

In [42]:
df.groupby('player_first_name').count()

,added,element_in,element_out,entry,event,id,index,kind,priority,result,entry_id
player_first_name,,,,,,,,,,,
Benji,30,30,30,30,30,30,21,30,21,30,30
Cory,30,30,30,30,30,30,22,30,22,30,30
Dave,18,18,18,18,18,18,13,18,13,18,18
James,25,25,25,25,25,25,19,25,19,25,25
John,40,40,40,40,40,40,30,40,30,40,40
Liam,7,7,7,7,7,7,7,7,7,7,7
Rebecca,12,12,12,12,12,12,12,12,12,12,12
Thomas,29,29,29,29,29,29,18,29,18,29,29
ben,21,21,21,21,21,21,20,21,20,21,21
